# Cross-Validation und Modellevaluierung

## Cross-Validation

In [ ]:
!pip install -q openai backoff gpt-cost-estimator

In [ ]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
import time
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from gpt_cost_estimator import CostEstimator
import openai
from openai import OpenAI
from google.colab import userdata
import backoff
import random
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Projekt_Mobilisierung/Data/anno_sample_df_final.csv', sep = ";")

In [ ]:
api_key_name = "xxx"
api_key = "xxx"

client = OpenAI(
    api_key=api_key
)

@CostEstimator()
def query_openai(model, messages, mock=True, completion_tokens=10):
    return client.chat.completions.create(
                      model=model,
                      messages=messages,
                      max_tokens=600)

@backoff.on_exception(backoff.expo, (openai.RateLimitError, openai.APIError))
def run_request(system_prompt, user_prompt, model, mock):
  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]

  return query_openai(
          model=model,
          messages=messages,
          mock=mock
        )

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(kf.split(df)):
    class_df = df.iloc[train_index]
    example_df = df.iloc[test_index]
    globals()[f'class_df_{i+1}'] = class_df
    globals()[f'example_df_{i+1}'] = example_df

for i in range(5):
    class_df = globals()[f'class_df_{i+1}']
    example_df = globals()[f'example_df_{i+1}']
    print(f"Split {i+1}:")
    print(f"Train Size (class_df): {len(class_df)}, Test Size (example_df): {len(example_df)}")
    print()

In [ ]:
examples_df = example_df_5

category_names = [
    "Politischer Inhalt/Info",
    "Negative Campaigning",
    "Selbstpräsentation unpolitisch",
    "Selbstpräsentation politisch",
    "Parteipräsentation unpolitisch",
    "Parteipräsentation politisch",
    "Interaktion",
    "Neutrale Infos zur Stimmabgabe",
    "Mitgliederinfo",
    "Fundraising",
    "Veranstaltungshinweis",
    "Wahlaufruf direkt",
    "Sonstiges"
]

def extract_categories(row):
    categories = [category_names[i] for i, cat in enumerate(row[6:19]) if cat == 1]
    return ", ".join(categories)

example_text = "\n\n".join(
    f"posttext: {row['posttext']}\nText im Bild: {row['ocr_text']}\nHashtags: {row['hashtags']}\nKategorien: {extract_categories(row)}"
    for index, row in examples_df.iterrows()
)

In [ ]:
len(example_text)

133429

In [ ]:
system_prompt = """
Du bist ein fortgeschrittenes KI-Modell zur Klassifikation. Deine Aufgabe ist es, Instagram-Posts von Kandidierenden aus dem vergangenen Landtagswahlkampf 2023 in Bayern nach den Kommunikationsstrategien zu klassifizieren. Ziel der Untersuchung ist es, Handlungsmuster in Wahlkampfposts zu quantifizieren und zu verstehen, welche Strategien mit Blick auf die Performance und Reichweite der Posts am besten funktionieren. Die Kodierungen helfen dabei zu verstehen, wie Wahlkampf in sozialen Medien verbessert werden kann und was das für demokratische Parteien bedeutet.

Bei der Klassifikation sollen die Beiträge in zwölf Kategorien klassifiziert werden. Mehrfachnennungen sind möglich, ein Post kann mehrere Handlungsmuster abbilden. Der theoretische Unterbau meiner Arbeit geht davon aus, dass Politiker:innen folgende zwölf Motive in sozialen Medien verfolgen:

1. **Politischer Inhalt/Info**: Politische Inhalte und Themen werden bearbeitet, oft verknüpft mit Forderungen oder wie man selbst oder die eigene Partei dazu steht. Weit gefasst, auch die Forderung nach einem „gerechterem Bayern“ reicht hier schon aus.
   - Beispiel: "Wir setzen uns für ein gerechteres Bayern ein, indem wir..."

2. **Negative Campaigning**: Die politische Konkurrenz wird diskreditiert und/oder vor allem kritisiert. Meist inhaltlich geprägt. Eher streng, es muss der Gegner namentlich genannt werden. Es müssen also andere Parteien (CSU, SPD, FDP, AFD, Grüne, Freie Wähler, Linke) oder die Staatsregierung in Bayern, die Bundesregierung (Ampel) in Deutschland oder die Opposition genannt werden.
   - Beispiel: "Die CSU hat in den letzten Jahren versagt, weil..."

3. **Selbstpräsentation unpolitisch**: Die eigene Person der Kandidierenden wird in unpolitischem Kontext präsentiert. Oft geht es darum, Nähe herzustellen und sich als „normale“ Person zu präsentieren. Oft werden positive Themen wie Ferien, Urlaub, Feiern oder schönes Wetter bedient.
   - Beispiel: "Ich genieße die Zeit mit meiner Familie im Urlaub."

4. **Selbstpräsentation politisch**: Die eigene Person wird in politischem Kontext präsentiert. Entweder die eigene politische Arbeit in Mandat, Partei, oder der Besuch von Institutionen im politischen Kontext. Auch die Präsentation der eigenen analogen Wahlkampfmethoden (Plakate, Infostand, Haustürbesuche, Podiumsdiskussionen) gehört hier dazu. Auch die Präsentation der eigenen Arbeit im vorpolitischen Raum (Vereine und Organisationen) gehört hier dazu. Wichtig ist hier, dass die Kategorie immer mit einem tatsächlichen Ereignis in der realen Welt verbunden ist, von der die Kandidierenden berichten und auch selbst dabei waren. Also der Besuch einer Firma vor Ort oder der Bericht von Haustürbesuchen. Die Präsentation der eigenen politischen Standpunkte fällt hier nicht mit hinein, das ist Kategorie 1.
   - Beispiel: "Heute habe ich die Firma XY besucht und mich über ihre Innovationen informiert."

5. **Parteipräsentation unpolitisch**: Die eigene Partei wird in unpolitischem Kontext präsentiert. Oft geht es darum, Nähe herzustellen und sich als normale Person zu präsentieren. Oft werden positive Themen wie Ferien, Urlaub, Feiern oder schönes Wetter bedient. Im Vergleich zu Kategorie 3 geht es hier um den Partei-Kontext, also ob die Themen und Veranstaltungen gemeinsam besucht/präsentiert werden. Typischerweise wird dann davon berichtet, dass Veranstaltungen gemeinsam besucht wurden (Stichwort „wir“) und es werden anderen Politiker:innen oder die Partei selbst und ihre Untergliederungen markiert mit dem @-Tag. Wenn Kategorie 5 gewählt wird, schließt das Kategorie 3 nicht aus, die Person des Kandidierenden wird dann auch immer mit präsentiert.
   - Beispiel: "Gemeinsam mit meinen Parteikolleg:innen genieße ich das schöne Wetter."

6. **Parteipräsentation politisch**: Die eigene Partei wird in politischem Kontext präsentiert. Entweder die eigene politische Arbeit in Mandat, Partei, oder der Besuch von Institutionen im politischen Kontext. Typisch ist auch der Besuch eines „Partei-VIPs“. Im Vergleich zu Kategorie 3 geht es hier um den Partei-Kontext, also ob die Themen und Veranstaltungen gemeinsam besucht/präsentiert werden. Zusätzlich zählt hierzu die Präsentation von Umfrageergebnissen. Typischerweise wird dann davon berichtet, dass Veranstaltungen gemeinsam besucht wurden (Stichwort „wir“) und es werden anderen Politiker:innen oder die Partei selbst und ihre Untergliederungen markiert mit dem @-Tag. Wenn Kategorie 6 gewählt wird, schließt das Kategorie 4 nicht aus, die Person des Kandidierenden wird dann auch immer mit präsentiert, da er zeigt, dass er in der Partei gut vernetzt ist und dort gute Arbeit leistet. Die Präsentation der parteipolitischen Standpunkte fällt hier nicht mit hinein, das ist Kategorie 1.
   - Beispiel: "Mit dem Parteivorsitzenden haben wir heute die neue Kampagne gestartet."

7. **Interaktion**: Instagram wird von den Kandidierenden genutzt, um mit dem Wahlvolk in Kontakt zu treten. Typisch sind Aufforderungen in die Kommentare zu schreiben oder sich per Direktnachricht zu melden. Auch Fragezeichen und Fragen zum Post bzw. zur Meinung der Menschen sind deutliche Hinweise für diese Kategorie. Die Interaktion muss aber immer im digitalen Raum stattfinden, die Aufforderung zum Wahlkampfstand vorbei zukommen zählt hier nicht.
   - Beispiel: "Was haltet ihr von unserem neuen Programm? Kommentiert unten!"

8. **Neutrale Infos zur Stimmabgabe**: Es wird neutral über den Ablauf der Stimmabgabe informiert, ohne für die eigene Person oder Partei zu werben. Ein Hinweis auf welcher Liste man selbst zu finden ist reicht hier nicht aus, es muss ein längerer detaillierter Text sein, der darauf hinweist, wie die Wahlabgabe im Detail funktioniert.
   - Beispiel: "So funktioniert die Briefwahl: Schritt 1..."

9. **Mitgliederinfo**: Es wird spezifisch mit den Mitgliedern der eigenen Partei/Organisation kommuniziert, um interne Themen zu bearbeiten.
   - Beispiel: "Liebe Mitglieder, bitte denkt an unsere nächste Versammlung am Freitag."

10. **Fundraising**: Es wird versucht, finanzielle Mittel für Wahlkampf und generell für die politische Arbeit einzutreiben, meistens als Spendenaufruf.
    - Beispiel: "Unterstützt unseren Wahlkampf mit einer Spende!"

11. **Veranstaltungshinweis**: Es wird auf eine (Wahlkampf)-Veranstaltung hingewiesen oder eingeladen. Die Veranstaltung muss in der Zukunft liegen, ein Bericht über eine vergangene Veranstaltung zählt hier nicht, das wäre Kategorie 4 und/oder 6.
    - Beispiel: "Kommt zu unserer Wahlkampfveranstaltung am Samstag!"

12. **Wahlaufruf direkt**: Es wird offen dazu aufgerufen, dass die eigene Partei oder Person gewählt werden soll. Oft verbunden mit Infos zum Ablauf der Wahl. Die reine Nennung auf welchem Listenplatz man steht reicht hier nicht aus.
    - Beispiel: "Wählt mich am 15. Oktober für ein besseres Bayern!"

13. **Sonstiges/KA**: Nur wenn keine der 12 Kategorien zutrifft, kann nur exklusiv ausgewählt werden. Es geht also nicht, dass 12 und eine andere Kategorie klassifiziert werden.

Hier sind einige klassifizierte Beispiele:
{example_text}

Bei der Klassifikation bekommst du den Text des Beitrags, eine Bildbeschreibung, die Schrift im Bild und die Hashtags zur Verfügung gestellt. Deine Aufgabe ist es zu entscheiden, ob die Strategien 1-12 im Post vorliegen. Gib die zutreffenden Kategorien als Liste zurück. Wenn keine der Kategorien zutrifft, gib "Sonstiges/KA" zurück, dass kommt allerdings kaum vor.

"""

In [ ]:
prompt = """
Bitte klassifiziere die Posts aus dem Datensatz anhand der Spalten „posttext“, „ocr_text“, "caption" und „hashtags“. Du kannst mehrere Kategorien als zutreffend klassifizieren (Multi-shot). Hier die Kategorien:
„1. Politischer Inhalt/Info“
„2. Negative Campaigning“
„3. Selbstpräsentation unpolitisch“
„4. Selbstpräsentation politisch“
„5. Parteipräsentation unpolitisch“
„6. Parteipräsentation politisch“
„7. Interaktion“
„8. Neutrale Infos zur Stimmabgabe“
„9. Mitgliederinfo“
„10. Fundraising“
„11. Veranstaltungshinweis“
„12. Wahlaufruf direkt“
„13. Sonstiges“
"Posttext: [posttext]\n"
"ocr_text: [ocr_text]\n"
"Caption: [caption]\n"
"Hashtags: [hashtags]"
"""

In [ ]:
CostEstimator.PRICES['gpt-4o-mini'] = {
    'input': 0.15/1000,
    'output': 0.60/1000
}

MOCK = True
RESET_COST = True
COLUMN = 'Kategorie'
SAMPLE_SIZE = 0
MODEL = "gpt-4o-mini"

if COLUMN not in class_df_5.columns:
    class_df_5[COLUMN] = None

categories = [
    "Politischer Inhalt/Info",
    "Negative Campaigning",
    "Selbstpräsentation unpolitisch",
    "Selbstpräsentation politisch",
    "Parteipräsentation unpolitisch",
    "Parteipräsentation politisch",
    "Interaktion",
    "Neutrale Infos zur Stimmabgabe",
    "Mitgliederinfo",
    "Fundraising",
    "Veranstaltungshinweis",
    "Wahlaufruf direkt",
    "Sonstiges"
]

for category in categories:
    if category not in class_df_5.columns:
        class_df_5[category] = 0

if RESET_COST:
    CostEstimator.reset()
    print("Reset Cost Estimation")

filtered_df = class_df_5.copy()

if not filtered_df.empty:
    if SAMPLE_SIZE > 0:
        SAMPLE_SIZE = min(SAMPLE_SIZE, len(filtered_df))
        filtered_df = filtered_df.sample(SAMPLE_SIZE)

    for index, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
        try:
            combined_text = (
                f"posttext: {str(row['posttext'])}\n"
                f"OCR_Text: {str(row['ocr_text'])}\n"
                f"Caption: {str(row['caption'])}\n"
                f"Hashtags: {str(row['hashtags'])}"
            )

            p = prompt.replace('[posttext]', str(row['posttext']))
            p = p.replace('[ocr_text]', str(row['ocr_text']))
            p = p.replace('[caption]', str(row['caption']))
            p = p.replace('[hashtags]', str(row['hashtags']))

            response = run_request(system_prompt, p, MODEL, MOCK)

            if not MOCK:
                response_content = response.choices[0].message.content

                for category in categories:
                    class_df_5.at[index, category] = 0

                for category in categories:
                    if category.lower() in response_content.lower():
                        class_df_5.at[index, category] = 1

        except Exception as e:
            import traceback
            print(f"An error occurred: {e}")
            traceback.print_exc()

else:
    print("No rows to process.")

print()

Reset Cost Estimation


<ipython-input-65-2d7bcc43a49e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_df_5[COLUMN] = None
<ipython-input-65-2d7bcc43a49e>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_df_5[category] = 0
<ipython-input-65-2d7bcc43a49e>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

  0%|          | 0/801 [00:00<?, ?it/s]

Cost: $0.0001 | Total: $0.1152


In [ ]:
classified_sample_df = class_df_5.loc[filtered_df.index]

In [ ]:
classified_sample_df.to_csv("/content/drive/MyDrive/Projekt_Mobilisierung/Data/class_cv_df_5.csv", index=False)

## Modellevaluation

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

true_labels_columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '99']
true_labels = classified_sample_df[true_labels_columns]
predictions = classified_sample_df[categories]

all_true = []
all_pred = []

for i, category in enumerate(categories):
    true_category = true_labels.iloc[:, i]
    pred_category = predictions[category]

    accuracy = accuracy_score(true_category, pred_category)
    precision = precision_score(true_category, pred_category, zero_division=0)
    recall = recall_score(true_category, pred_category, zero_division=0)
    f1 = f1_score(true_category, pred_category, zero_division=0)

    print(f"Category: {category}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print()

    all_true.extend(true_category)
    all_pred.extend(pred_category)

overall_accuracy_2 = accuracy_score(all_true, all_pred)
overall_precision_2 = precision_score(all_true, all_pred, average='weighted', zero_division=0)
overall_recall_2 = recall_score(all_true, all_pred, average='weighted', zero_division=0)
overall_f1_2 = f1_score(all_true, all_pred, average='weighted', zero_division=0)

print("Overall Metrics:")
print(f"Accuracy: {overall_accuracy_2}")
print(f"Precision: {overall_precision_2}")
print(f"Recall: {overall_recall_2}")
print(f"F1-Score: {overall_f1_2}")

In [ ]:
overall_precision_macro = precision_score(all_true, all_pred, average='macro', zero_division=0)
overall_recall_macro = recall_score(all_true, all_pred, average='macro', zero_division=0)
overall_f1_macro = f1_score(all_true, all_pred, average='macro', zero_division=0)

print("Overall Macro Metrics:")
print(f"Precision (Macro): {overall_precision_macro}")
print(f"Recall (Macro): {overall_recall_macro}")
print(f"F1-Score (Macro): {overall_f1_macro}")

In [ ]:
metrics_df = pd.DataFrame({
    "Overall Accuracy": [overall_accuracy_2],
    "Overall Precision": [overall_precision_2],
    "Overall Recall": [overall_recall_2],
    "Overall F1-Score": [overall_f1_2]
})

file_path = "/content/drive/MyDrive/Projekt_Mobilisierung/Data/overall_metrics.csv"
metrics_df.to_csv(file_path, index=False)

print(f"Metrics saved to {file_path}")

Metrics saved to /content/drive/MyDrive/Projekt_Mobilisierung/Data/overall_metrics.csv


In [ ]:
file_path = "/content/drive/MyDrive/Projekt_Mobilisierung/Data/overall_metrics.csv"

try:
    metrics_df = pd.read_csv(file_path)
except FileNotFoundError:
    metrics_df = pd.DataFrame(columns=["Overall Accuracy", "Overall Precision", "Overall Recall", "Overall F1-Score"])

new_metrics = pd.DataFrame({
    "Overall Accuracy": [overall_accuracy_2],
    "Overall Precision": [overall_precision_2],
    "Overall Recall": [overall_recall_2],
    "Overall F1-Score": [overall_f1_2]
})

metrics_df = pd.concat([metrics_df, new_metrics], ignore_index=True)

metrics_df.to_csv(file_path, index=False)

print(f"Metrics updated and saved to {file_path}")

Metrics updated and saved to /content/drive/MyDrive/Projekt_Mobilisierung/Data/overall_metrics.csv


In [ ]:
metrics = pd.read_csv("/content/drive/MyDrive/Projekt_Mobilisierung/Data/overall_metrics.csv")

In [ ]:
mean_values = metrics.mean()

metrics.loc['Mean'] = mean_values

print(metrics)

file_path = "/content/drive/MyDrive/Projekt_Mobilisierung/Data/overall_metrics.csv"
metrics.to_csv(file_path, index=False)

      Overall Accuracy  Overall Precision  Overall Recall  Overall F1-Score
0             0.884712           0.907369        0.884712          0.892810
1             0.880150           0.904123        0.880150          0.888787
2             0.880054           0.906044        0.880054          0.889232
3             0.884183           0.907797        0.884183          0.892583
4             0.882647           0.905766        0.882647          0.890936
Mean          0.882349           0.906220        0.882349          0.890869


Quellen:

Achmann-Denkler, M. (2024). michaelachmann/social-media-lab: DOI Release (v0.0.12). Zenodo. https://doi.org/10.5281/zenodo.10618621

Achmann-Denkler, M. (2024). “Text Classification.” January 22, 2024. https://doi.org/10.5281/zenodo.10039756.